# Explore here

It's recommended to use this notebook for exploration purposes.

For example: 

1. You could import the CSV generated by python into your notebook and explore it.
2. You could connect to your database using `pandas.read_sql` from this notebook and explore it.

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import pickle

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb

In [2]:
df_raw = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/random-forest-project-tutorial/main/titanic_train.csv',index_col=0)

In [3]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [4]:
df_raw.sample(10)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
671,1,2,"Brown, Mrs. Thomas William Solomon (Elizabeth ...",female,40.0,1,1,29750,39.0000,NaN,S
618,0,3,"Lobb, Mrs. William Arthur (Cordelia K Stanlick)",female,26.0,1,0,A/5. 3336,16.1000,NaN,S
507,1,2,"Quick, Mrs. Frederick Charles (Jane Richards)",female,33.0,0,2,26360,26.0000,NaN,S
682,1,1,"Hassab, Mr. Hammad",male,27.0,0,0,PC 17572,76.7292,D49,C
584,0,1,"Ross, Mr. John Hugo",male,36.0,0,0,13049,40.1250,A10,C
631,1,1,"Barkworth, Mr. Algernon Henry Wilson",male,80.0,0,0,27042,30.0000,A23,S
729,0,2,"Bryhl, Mr. Kurt Arnold Gottfrid",male,25.0,1,0,236853,26.0000,NaN,S
886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,NaN,Q
245,0,3,"Attalah, Mr. Sleiman",male,30.0,0,0,2694,7.2250,NaN,C


In [5]:
df_raw.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [18]:
df_raw.describe(include='object')

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Braund, Mr. Owen Harris",male,347082,B96 B98,S
freq,1,577,7,4,644


### Hago drop a los datos de la cabina y en los faltantes de la edad le completo con la media 

In [3]:
df_raw = df_raw.drop(columns='Cabin', axis=1)

In [4]:
df_raw['Age'] = df_raw['Age'].fillna(df_raw['Age'].mean())
df_raw['Embarked'] = df_raw['Embarked'].fillna(df_raw['Embarked'].mode()[0])

In [8]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       891 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 76.6+ KB


In [5]:
df = df_raw.copy()

In [6]:
X = df.drop(columns=['Ticket', 'Name', 'Survived'])
y= df['Survived']

In [7]:
X [['Sex', 'Embarked']] = X [['Sex', 'Embarked']].astype('category')

In [8]:
X['Sex'] = X['Sex'].cat.codes

In [9]:
X['Embarked'] = X['Embarked'].cat.codes

In [25]:
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
1,3,1,22.0,1,0,7.2500,2
2,1,0,38.0,1,0,71.2833,0
3,3,0,26.0,0,0,7.9250,2
4,1,0,35.0,1,0,53.1000,2
5,3,1,35.0,0,0,8.0500,2


In [10]:
X_train, X_test, y_train, y_test = train_test_split (X, y, random_state=1107)

In [11]:
classifer = RandomForestClassifier()

In [12]:
classifer.fit(X_train, y_train)

RandomForestClassifier()

In [13]:
y_pred = classifer.predict(X_test)

In [14]:
print(classification_report (y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.89      0.87       141
           1       0.79      0.72      0.75        82

    accuracy                           0.83       223
   macro avg       0.82      0.80      0.81       223
weighted avg       0.82      0.83      0.82       223



### Buscamos los mejores Hiperparámetros

In [16]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt'] # común es auto, sqrt le hace al valor que le pase

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Criterio
criterion = ['gini', 'entropy']

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               #'max_features': max_features, #son pocas variables
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'criterion': criterion}

print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False], 'criterion': ['gini', 'entropy']}


In [17]:
classifer_grid = RandomForestClassifier(random_state=1107)
classifer_grid_random = RandomizedSearchCV (estimator=classifer_grid, n_iter=100, cv=5, random_state=1107, param_distributions= random_grid)

In [18]:
classifer_grid_random.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=1107),
                   n_iter=100,
                   param_distributions={'bootstrap': [True, False],
                                        'criterion': ['gini', 'entropy'],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=1107)

#### El mejor modelo es

In [20]:
classifer_grid_random.best_params_

{'n_estimators': 400,
 'min_samples_split': 10,
 'min_samples_leaf': 2,
 'max_depth': 90,
 'criterion': 'entropy',
 'bootstrap': True}

In [21]:
classifer_with_grid = RandomForestClassifier(n_estimators= 400, min_samples_split= 10, min_samples_leaf = 2, max_depth = 90, criterion= 'entropy', bootstrap= True)

In [22]:
classifer_with_grid.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=90, min_samples_leaf=2,
                       min_samples_split=10, n_estimators=400)

In [23]:
y_pred_with_grid = classifer_with_grid.predict(X_test)

In [24]:
print(classification_report (y_test, y_pred_with_grid))

              precision    recall  f1-score   support

           0       0.85      0.94      0.89       141
           1       0.87      0.71      0.78        82

    accuracy                           0.85       223
   macro avg       0.86      0.82      0.83       223
weighted avg       0.85      0.85      0.85       223



In [52]:
mse=mean_squared_error(y_test, y_pred_with_grid)
print(np.sqrt(mse))

0.3846843677357699


In [25]:
print(classification_report (y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.89      0.87       141
           1       0.79      0.72      0.75        82

    accuracy                           0.83       223
   macro avg       0.82      0.80      0.81       223
weighted avg       0.82      0.83      0.82       223



In [27]:
clf_GB = GradientBoostingClassifier(n_estimators=100)
clf_GB.fit(X_train, y_train)
y_pred_GB = clf_GB.predict(X_test)
print(classification_report (y_test, y_pred_GB))

              precision    recall  f1-score   support

           0       0.83      0.90      0.86       141
           1       0.80      0.68      0.74        82

    accuracy                           0.82       223
   macro avg       0.82      0.79      0.80       223
weighted avg       0.82      0.82      0.82       223



In [51]:
mse=mean_squared_error(y_test, y_pred_GB)
print(np.sqrt(mse))

0.4235235498876267


In [41]:
xgb_model = xgb.XGBRegressor(objective="reg:linear", random_state=42)

In [42]:
xgb_model.fit(X_train, y_train)

[22:38:49] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, objective='reg:linear', predictor='auto',
             random_state=42, reg_alpha=0, ...)

In [43]:
y_pred_XG = xgb_model.predict(X_test)

In [47]:
y_pred_XG

array([ 4.40178275e-01,  1.16873991e+00,  1.00126529e+00, -5.60231805e-02,
        8.25704336e-02,  1.42413182e-02,  9.60491300e-01,  2.13323131e-01,
        8.70186090e-02,  5.06416321e-01,  7.64368251e-02, -5.97721385e-03,
        1.96880832e-01,  8.54336247e-02,  1.00068164e+00,  8.85704458e-01,
        9.75107551e-01,  9.69643176e-01,  8.30960035e-01,  4.28198844e-01,
       -9.04213861e-02,  9.81333137e-01,  1.16421683e-02,  6.80842340e-01,
        8.23144495e-01,  3.31337638e-02,  1.53230708e-02,  1.01179552e+00,
       -4.92111072e-02,  6.11733615e-01,  1.24310374e+00,  1.32474992e-02,
        1.13771605e+00,  9.28117692e-01,  2.09718004e-01,  9.21930015e-01,
       -4.59244251e-02, -9.30616558e-02,  5.76994598e-01,  5.20765722e-01,
       -6.32181168e-02,  7.40128338e-01,  7.26835728e-01,  1.02819729e+00,
        1.03087890e+00,  1.47239164e-01,  7.99550340e-02,  2.68668324e-01,
        7.65982270e-01,  1.59022398e-02,  7.04721689e-01, -2.34247441e-03,
        1.00624382e-01,  

In [49]:
mse=mean_squared_error(y_test, y_pred_XG)
print(np.sqrt(mse))

0.43419744585524905


In [25]:
# Guardo modelo en carpeta
filename = '../models/finalized_model.sav'
pickle.dump(classifer_with_grid, open(filename, 'wb'))